In [72]:
import tensorflow as tf
import numpy as np

In [ ]:
# learning rate decay

# initial_learning_rate = 0.1
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=100000,
#     decay_rate=0.96,
#     staircase=True)

# optimizer = keras.optimizers.RMSprop(learning_rate=lr_schedule)

# ExponentialDecay, PiecewiseConstantDecay, PolynomialDecay and InverseTimeDecay
# 여기에서 staircase는 learning rate를 계단식으로 감소시켜준다. 즉 decay_steps에 도달하기 전에는 learning rate를 유지시키다가, 
# decay_steps가 넘어가면 그 때 학습률을 감소시키는 것이다.


# def decayed_learning_rate(step):
#   return initial_learning_rate * decay_rate ^ (step / decay_steps)
# if staircase = True : step/decay_steps : integer division

In [ ]:
def exponential_decay(epoch) :
  initial_rate = 0.01
  k = 0.96
  exp_rate = initial_rate * np.exp(-k * t)

In [ ]:
## Feature Scaling

# sd = sqrt(np.sum((data - np.mean(data))^2) / np.count/(data))
# standardization = (data - np.mean(data)) / sd

# normalization = (data - np.min(data, axis = 0)) / (np.max(data, 0) - np.min(data, 0))

In [73]:
xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
               [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
               [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
               [816, 820.958984, 1008100, 815.48999, 819.23999],
               [819.359985, 823, 1188100, 818.469971, 818.97998],
               [819, 823, 1198100, 816, 820.450012],
               [811.700012, 815.25, 1098100, 809.780029, 813.669983],
               [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

## 성격이 다른 데이터 몇 개가 보임 ex) 908100


def normalization(data) :
  numerator = data - np.min(data,0)
  denominator = np.max(data,0) - np.min(data,0)
  return numerator / denominator


xy = normalization(xy)
xy = xy.astype(np.float32)

x_train = xy[:, 0:-1]
y_train = xy[:,[-1]]

In [74]:
dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(len(x_train))

W = tf.Variable(tf.random.normal([4,1]),dtype=tf.float32)
b = tf.Variable(tf.random.normal([1]),dtype=tf.float32)

In [75]:
def linearReg_fn(features) :
  hypothesis = tf.matmul(features, W) + b
  return hypothesis

def L2_loss(loss, beta = 0.01) :
  W_reg = tf.nn.l2_loss(W)  #output = sum(t ** 2) / 2
  loss = tf.reduce_mean(loss + W_reg * beta)
  return loss 

def loss_fn(hypothesis, labels, flag = False) :
  cost = tf.reduce_mean(tf.square(hypothesis - labels))
  if(flag) :
    cost = L2_loss(cost)
  return cost


In [76]:
is_decay = True
starter_learning_rate = 0.1

if(is_decay) :
  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    starter_learning_rate,
    decay_steps=50,
    decay_rate=0.96,
    staircase=True)
  optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)
else :
  optimizer = tf.keras.optimizers.SGD(learning_rate = starter_learning_rate)

In [77]:
def grad(features, labels, l2_flag) :
  with tf.GradientTape() as tape :
    loss_value = loss_fn(linearReg_fn(features),labels, l2_flag)
  return tape.gradient(loss_value, [W,b]), loss_value

In [79]:
for step in range(151) :
  for features, labels in dataset :
    features = tf.cast(features, tf.float32)
    labels = tf.cast(labels, tf.float32)
    grads, loss_value = grad(features, labels, False)
    optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
    current_learning_rate = optimizer._decayed_lr(tf.float32).numpy()
    if step % 10 == 0 :
      print("Iter : {}, Loss : {:.4f}, lr : {}".format(step,loss_value,current_learning_rate))


Iter : 0, Loss : 3.6949, lr : 0.10000000149011612
Iter : 10, Loss : 0.3817, lr : 0.10000000149011612
Iter : 20, Loss : 0.1694, lr : 0.10000000149011612
Iter : 30, Loss : 0.0543, lr : 0.10000000149011612
Iter : 40, Loss : 0.0110, lr : 0.10000000149011612
Iter : 50, Loss : 0.0574, lr : 0.09600000083446503
Iter : 60, Loss : 0.0038, lr : 0.09600000083446503
Iter : 70, Loss : 0.0335, lr : 0.09600000083446503
Iter : 80, Loss : 0.0269, lr : 0.09600000083446503
Iter : 90, Loss : 0.0147, lr : 0.09600000083446503
Iter : 100, Loss : 0.0216, lr : 0.09216000139713287
Iter : 110, Loss : 0.0292, lr : 0.09216000139713287
Iter : 120, Loss : 0.0191, lr : 0.09216000139713287
Iter : 130, Loss : 0.0195, lr : 0.09216000139713287
Iter : 140, Loss : 0.0226, lr : 0.09216000139713287
Iter : 150, Loss : 0.0221, lr : 0.0884735956788063


In [13]:
tf.cast(x_train, tf.float32)

# tf.cast 함수는 텐서를 새로운 자료형으로 변환합니다.
# Boolean형태인 경우 True이면 1, False이면 0을 출력한다.

<tf.Tensor: shape=(8, 4), dtype=float32, numpy=
array([[8.28660e+02, 8.33450e+02, 9.08100e+05, 8.28350e+02],
       [8.23020e+02, 8.28070e+02, 1.82810e+06, 8.21655e+02],
       [8.19930e+02, 8.24400e+02, 1.43810e+06, 8.18980e+02],
       [8.16000e+02, 8.20959e+02, 1.00810e+06, 8.15490e+02],
       [8.19360e+02, 8.23000e+02, 1.18810e+06, 8.18470e+02],
       [8.19000e+02, 8.23000e+02, 1.19810e+06, 8.16000e+02],
       [8.11700e+02, 8.15250e+02, 1.09810e+06, 8.09780e+02],
       [8.09510e+02, 8.16660e+02, 1.39810e+06, 8.04540e+02]],
      dtype=float32)>

In [69]:
# current_learning_rate = optimizer._decayed_lr(tf.float32)
# print(current_learning_rate)

# To print current learning rate

tf.Tensor(0.088473596, shape=(), dtype=float32)
